In [18]:
import h2o as h
import pandas as pd
h.init()
from h2o.estimators import H2OGradientBoostingEstimator
import treelite
import treelite.frontend
import numpy as np




Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,22 hours 15 mins
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_home_67pta2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.674 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
df = pd.read_excel('alldata.xlsx')
df.to_csv('alldata.csv', index=False)

In [19]:
data = h.import_file("alldata.csv")



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [20]:
train, valid = data.split_frame(ratios=[0.8], seed=123)
predictors = ['CGM_Data', 'carbs', 'calories', 'fiber', 'fat', 'protein','Gender (Female=1, Male=2)','Age (years)','BMI (kg/m2)','Weight (kg)','Hour','Duration of diabetes (years)']
response = 'avgCGM'

# Train GBM model
gbm = H2OGradientBoostingEstimator(ntrees=100, max_depth=3)
gbm.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

# View model performance
print(gbm)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1680506826855_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         14809                  3            3            3             4             8             7.14

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 739.5846897934606
RMSE: 27.195306392711604
MAE: 21.24741656064987
RMSLE: 0.17963596756039887
Mean Residual Deviance: 739.5846897934606

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 1783.048359421686
RMSE: 42.22615728931164
MAE

In [11]:
model_path = gbm.download_mojo(path="my_model.mojo")

## Converting to tflite

In [12]:
h2o_model = h.import_mojo(model_path)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [16]:
h2o_model = h.import_mojo(model_path)
predictor = h2o_model.predict
input_shape = [(None, 3)]
model = treelite_runtime.Model.from_xgboost(predictor, input_shape=input_shape)


generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


AttributeError: module 'treelite_runtime' has no attribute 'Model'

In [ ]:
compiler = treelite.compiler.Compiler(backend="tflite")
tflite_model = compiler.compile(model)

In [ ]:
with open("my_model.tflite", "wb") as f:
    f.write(tflite_model)

In [6]:
data = pd.read_csv('alldata.csv')
input_shape = [(None, data.shape[1])]


In [ ]:
model_path = h.save_model(model=gbm, path="my_model", force=True)


In [ ]:
h2o_model = h.import_mojo(model_path)
predictor = h2o_model.predict
input_shape = [(None, 3)]
model = treelite.frontend.from_h2o(predictor, input_shape=input_shape)


In [ ]:
compiler = treelite.compiler.Compiler(backend="tflite")
model = compiler.compile(model)
with open("my_model.tflite", "wb") as f:
    f.write(model)